# Introduction to Lance DB

In [1]:
import lancedb

db = lancedb.connect(uri = "vector_databse")
db

LanceDBConnection(uri='/Users/john.sandsjo/Documents/github/data_platform_ai_course/video_lecture/09_vector_db_intro/vector_databse')

In [2]:
db.uri

'/Users/john.sandsjo/Documents/github/data_platform_ai_course/video_lecture/09_vector_db_intro/vector_databse'

## Create a table

In [6]:
import json

with open("data/animals_text_embeddings.json", "r") as file:
    data = json.loads(file.read())
data

[{'text': 'A small brown dog running.', 'vector': [0.12, 0.85, 0.33]},
 {'text': 'A cat resting quietly on a sofa.', 'vector': [0.4, 0.91, 0.1]},
 {'text': 'A large gray elephant drinking water.',
  'vector': [0.88, 0.22, 0.55]},
 {'text': 'A fast cheetah sprinting across the savannah.',
  'vector': [0.95, 0.12, 0.72]},
 {'text': 'A colorful parrot perched on a branch.',
  'vector': [0.25, 0.66, 0.81]},
 {'text': 'A frog sitting on a lily pad.', 'vector': [0.14, 0.44, 0.27]}]

In [ ]:
table_animals = db.create_table("animals_text", exist_ok=True, data = data)
#mode = "overwrite" skriver över
table_animals

LanceTable(name='animals_text', version=1, _conn=LanceDBConnection(uri='/Users/john.sandsjo/Documents/github/data_platform_ai_course/video_lecture/09_vector_db_intro/vector_databse'))

In [8]:
table_animals.head()

pyarrow.Table
text: string
vector: fixed_size_list<item: float>[3]
  child 0, item: float
----
text: [["A small brown dog running.","A cat resting quietly on a sofa.","A large gray elephant drinking water.","A fast cheetah sprinting across the savannah.","A colorful parrot perched on a branch."]]
vector: [[[0.12,0.85,0.33],[0.4,0.91,0.1],[0.88,0.22,0.55],[0.95,0.12,0.72],[0.25,0.66,0.81]]]

In [9]:
table_animals.to_pandas()

,text,vector
0,A small brown dog running.,"[0.12, 0.85, 0.33]"
1,A cat resting quietly on a sofa.,"[0.4, 0.91, 0.1]"
2,A large gray elephant drinking water.,"[0.88, 0.22, 0.55]"
3,A fast cheetah sprinting across the savannah.,"[0.95, 0.12, 0.72]"
4,A colorful parrot perched on a branch.,"[0.25, 0.66, 0.81]"
5,A frog sitting on a lily pad.,"[0.14, 0.44, 0.27]"


In [10]:
more_data = [
    {"text": "A panda eating bamboo peacefully.", "vector": [0.51, 0.37, 0.82]},
    {"text": "A lion roaring loudly on a rock.", "vector": [0.93, 0.18, 0.41]},
]

table_animals.add(more_data)

AddResult(version=2)

In [11]:
table_animals.to_pandas()

,text,vector
0,A small brown dog running.,"[0.12, 0.85, 0.33]"
1,A cat resting quietly on a sofa.,"[0.4, 0.91, 0.1]"
2,A large gray elephant drinking water.,"[0.88, 0.22, 0.55]"
3,A fast cheetah sprinting across the savannah.,"[0.95, 0.12, 0.72]"
4,A colorful parrot perched on a branch.,"[0.25, 0.66, 0.81]"
5,A frog sitting on a lily pad.,"[0.14, 0.44, 0.27]"
6,A panda eating bamboo peacefully.,"[0.51, 0.37, 0.82]"
7,A lion roaring loudly on a rock.,"[0.93, 0.18, 0.41]"


## create an empty table and delete it

In [12]:
from lancedb.pydantic import LanceModel

#A Pydantic Model base class that can be converted to a LanceDB Table.
class JokesSchema(LanceModel):
    joke: str
    rating: int

db.create_table(name= "jokes", schema= JokesSchema, exist_ok=True)

LanceTable(name='jokes', version=1, _conn=LanceDBConnection(uri='/Users/john.sandsjo/Documents/github/data_platform_ai_course/video_lecture/09_vector_db_intro/vector_databse'))

In [14]:
db.table_names()

['animals_text', 'jokes']

In [15]:
db.drop_table("jokes")

In [16]:
db.table_names()

['animals_text']

## Open existing table

In [18]:
db.open_table("animals_text").head(2)

pyarrow.Table
text: string
vector: fixed_size_list<item: float>[3]
  child 0, item: float
----
text: [["A small brown dog running.","A cat resting quietly on a sofa."]]
vector: [[[0.12,0.85,0.33],[0.4,0.91,0.1]]]

## Vector search in Lance DB

In [19]:
table_animals.to_pandas()

,text,vector
0,A small brown dog running.,"[0.12, 0.85, 0.33]"
1,A cat resting quietly on a sofa.,"[0.4, 0.91, 0.1]"
2,A large gray elephant drinking water.,"[0.88, 0.22, 0.55]"
3,A fast cheetah sprinting across the savannah.,"[0.95, 0.12, 0.72]"
4,A colorful parrot perched on a branch.,"[0.25, 0.66, 0.81]"
5,A frog sitting on a lily pad.,"[0.14, 0.44, 0.27]"
6,A panda eating bamboo peacefully.,"[0.51, 0.37, 0.82]"
7,A lion roaring loudly on a rock.,"[0.93, 0.18, 0.41]"


In [20]:
query_vector = [0.5, 0.2, 0.9]

table_animals.search(query_vector).limit(3).to_pandas()

,text,vector,_distance
0,A panda eating bamboo peacefully.,"[0.51, 0.37, 0.82]",0.0354
1,A fast cheetah sprinting across the savannah.,"[0.95, 0.12, 0.72]",0.2413
2,A large gray elephant drinking water.,"[0.88, 0.22, 0.55]",0.2673


## Embedding API

- idea: want t put in text -> and it will automatically generate vector embeddings
- put in a query and it will automagically generate vector embeddings
- claculate closest distances

In [21]:
from lancedb.pydantic import LanceModel, Vector
from lancedb.embeddings import get_registry

model = get_registry().get("gemini-text").create(name = "gemini-embedding-001")
model

GeminiText(max_retries=7, name='gemini-embedding-001', query_task_type='retrieval_query', source_task_type='retrieval_document')

In [23]:
import numpy as np
hello_embedding = np.array(model.generate_embeddings("hello"))
hello_embedding.shape

(5, 3072)

In [ ]:
class JokeModel(LanceModel):
    joke: str = model.SourceField() # joke becomes our source column
    vector: Vector(3072) = model.VectorField() #target vector column

table_joke = db.create_table("jokes", schema=JokeModel, exist_ok=True)
table_joke

LanceTable(name='jokes', version=1, _conn=LanceDBConnection(uri='/Users/john.sandsjo/Documents/github/data_platform_ai_course/video_lecture/09_vector_db_intro/vector_databse'))

In [27]:
import pandas as pd
with open("data/jokes.json", "r") as file:
    jokes_data = json.loads(file.read())

df_jokes = pd.DataFrame(jokes_data).rename({"jokes": "joke"}, axis=1)
df_jokes.head()

,joke
0,Parallel lines have so much in common—it’s sad...
1,"ETL stands for “Extract, Transform, Leave for ..."
2,What do you call a snake that runs your script...
3,"Gold walks into a bar. The bartender says, “Au..."
4,C# devs don’t argue; they just throw exceptions.


In [28]:
df_jokes["joke"].iloc[0]

'Parallel lines have so much in common—it’s sad they’ll never meet.'

### add data to table

In [29]:
table_joke.add(df_jokes)

AddResult(version=2)

In [30]:
table_joke.head()

pyarrow.Table
joke: string not null
vector: fixed_size_list<item: float>[3072]
  child 0, item: float
----
joke: [["Parallel lines have so much in common—it’s sad they’ll never meet.","ETL stands for “Extract, Transform, Leave for the next person.”","What do you call a snake that runs your scripts? A py-thon (ton of errors).","Gold walks into a bar. The bartender says, “Au, get out!”","C# devs don’t argue; they just throw exceptions."]]
vector: [[[-0.024001757,0.01247358,-0.024144737,-0.06704516,0.017059995,...,0.011402721,-0.017770408,0.011606138,0.0004488597,0.012175956],[-0.015356114,0.0211365,-0.021389864,-0.07957475,0.008829045,...,0.023671096,0.00070166244,0.003873497,0.0006493248,0.01587487],[-0.01761013,0.0031474787,-0.015632002,-0.060952485,0.003696307,...,0.015392491,-0.00025712984,0.0064440705,0.0017164479,-0.00869067],[-0.024867292,0.013314825,-0.016261652,-0.07383461,0.0085606,...,0.031690735,-0.0027016324,0.017647061,0.008275158,-0.0067860777],[-0.0068662865,-0.005415149,

In [32]:
table_joke.to_pandas()["vector"][0]

array([-0.02400176,  0.01247358, -0.02414474, ...,  0.01160614,
        0.00044886,  0.01217596], shape=(3072,), dtype=float32)

## Perform vector search

In [33]:
table_joke.search("data engineering jokes").limit(5).to_pandas()

,joke,vector,_distance
0,Why do data engineers hate nature? Too many un...,"[-0.027916763, 0.0047387416, -0.018934403, -0....",0.461179
1,"Data engineer motto: If it works, don’t touch ...","[-0.020296954, 0.020327171, -0.009069326, -0.0...",0.542293
2,What do you call a snake that runs your script...,"[-0.01761013, 0.0031474787, -0.015632002, -0.0...",0.655434
3,"Gold walks into a bar. The bartender says, “Au...","[-0.024867292, 0.013314825, -0.016261652, -0.0...",0.667807
4,I tried to explain async/await to my friend… n...,"[-0.016174542, 0.012602222, -0.015254588, -0.0...",0.679036


In [34]:
table_joke.search("give me chemistry related jokes").limit(5).to_pandas()

,joke,vector,_distance
0,I told a chemistry joke… there was no reaction.,"[-0.022922393, 0.017959604, -0.029222224, -0.0...",0.576598
1,Why did the chemist ground his kids? Because t...,"[-0.023257235, 0.016145445, -0.029016329, -0.0...",0.613355
2,"Gold walks into a bar. The bartender says, “Au...","[-0.024867292, 0.013314825, -0.016261652, -0.0...",0.671744
3,Chemists have all the solutions… mostly in bea...,"[-0.012411982, -0.002532549, -0.0061755716, -0...",0.673519
4,Oxygen and Magnesium got together—OMg!,"[-0.024720555, 0.006975079, -0.015012385, -0.0...",0.738241


In [35]:
table_joke.search("give me some nature related search").limit(5).to_pandas()

,joke,vector,_distance
0,Why do data engineers hate nature? Too many un...,"[-0.027916763, 0.0047387416, -0.018934403, -0....",0.691816
1,I asked the data lake if it had my file. It sa...,"[-0.026169129, 0.01796462, -0.013938847, -0.09...",0.770659
2,Oxygen and Magnesium got together—OMg!,"[-0.024720555, 0.006975079, -0.015012385, -0.0...",0.798217
3,"Dear algebra, please stop asking us to find yo...","[-0.034297567, 0.012844127, -0.02204103, -0.06...",0.808556
4,Chemists have all the solutions… mostly in bea...,"[-0.012411982, -0.002532549, -0.0061755716, -0...",0.812397


### hybrid search

combines traditonal keyword based search with vector similiraty search

In [ ]:
# to enable keyword search, we need to create an in index on the joke column

table_joke.create_fts_index("joke", replace=True)

In [38]:
from lancedb import rerankers

reranker = rerankers.RRFReranker()

results = table_joke.search(
    "give me nature related jokes",
    query_type = "hybrid",
    vector_column_name = "vector",
    fts_columns="joke"
).rerank(reranker=reranker).limit(5).to_pandas()

results

,joke,vector,_relevance_score
0,Why do data engineers hate nature? Too many un...,"[-0.027916763, 0.0047387416, -0.018934403, -0....",0.032787
1,"Gold walks into a bar. The bartender says, “Au...","[-0.024867292, 0.013314825, -0.016261652, -0.0...",0.016129
2,Why did the chemist ground his kids? Because t...,"[-0.023257235, 0.016145445, -0.029016329, -0.0...",0.015873
3,I told a chemistry joke… there was no reaction.,"[-0.022922393, 0.017959604, -0.029222224, -0.0...",0.015625
4,The C# compiler walked into a bar. The bartend...,"[-0.01868987, 0.018796643, -0.009748903, -0.07...",0.015385


## some rule of thumb for hybrid searc

- exact matching -> FTS full text search
- meaning based matching -> vector search
- both, unpredictable or mixed queries -> hybrid search